# Singular Value Decomposition

Let $A$ be a real $m\times n$ matrix with $m\geqslant n$. It is well known that 

$$A = U\mathit{\Sigma}V^T~~~~~~~~~~~~(1)$$


where

$$U^TU = V^TV = VV^T = I_n ~ and ~ \mathit{\Sigma} = diag (\sigma_1, ... , \sigma_n).$$

The matrix $U$ consists of $n$ orthonormalized eigenvectors associated with the $n$ largest eigenvalues of $AA^T$, and the matrix $V$ consists of the orthonormalized eigenvectors of $A^TA$. The diagonal elements of $\mathit{\Sigma}$ are the non-negative square roots of the eigenvalues of $A^TA$; they are called $\textit{singular values}$. We shall assume that

$$\sigma_1\geqslant~...~\geqslant \sigma_n\geqslant 0.$$

Thus if $rank($A$)=r$, $\sigma_{r+1}=~...~= \sigma_{n}=0$. The decomposition (1) is called the $\textit{singular value decomposition (SVD)}$. 

If the matrix $U$ is not needed, it would appear that one could apply the usual diagonatization algorithms to the symmetric matrix $A^TA$ which has to be formed explicitly. However, as in the case of linear least squares problems, the computation of $A^TA$ involves unnecessary numerical inaccuracy. For example, let 

$$\mathbf{A} = \begin{bmatrix}
            \mathbf{1} & \mathbf{1}\\
            \mathbf{\beta} & \mathbf{0}\\
            \mathbf{0} & \mathbf{\beta}
        \end{bmatrix},$$
        
then

$$\mathbf{A^TA} = \begin{bmatrix}
            \mathbf{1+\beta^2} & \mathbf{1}\\
            \mathbf{1} & \mathbf{1+\beta^2}\\
        \end{bmatrix}$$
        
so that

$$\sigma_1(A)=\sqrt{(2+\beta^2)},~~~~~\sigma_2(A)=|\beta|.$$

If $\beta^2 < \varepsilon_0$, the machine precision, the computed $A^TA$ has the form $\begin{bmatrix} 1 & 1 \\ 1 & 1 \end{bmatrix}$, and the best one may obtain from diagonalization is $\hat\sigma_1=\sqrt2 , \hat\sigma_2=0$ . 

The program described below is a two-step algorithm. It first reduces the matrix to bidiagonal form and then finds the SVD of the bidiagonal matrix. Reduction to bidiagonal form is accomplished using Householder transformations. Finding the SVD of a bidiagonal matrix is an iterative process that must be carefully performed in order to minimize both numerical errors and the number of iterations required.

In [1]:
import numpy as np
from numpy.testing import assert_allclose

TOL = 1e-4

In [2]:
def get_sign(x):
    if x >= 0:
        return 1
    return -1

In [3]:
def set_lowVal_zero(X):
    
    low_values_indices = abs(X) < 9e-15   # where values are low
    X[low_values_indices] = 0  # all low values set to 0
    
    return X

In [4]:
def householder(vec, i):
    x = np.asarray(vec, dtype=float)
    if x.ndim != 1:
        raise ValueError("vec.ndim = %s, expected 1" % x.ndim)
        
    y = x.copy()
    y[i] = np.linalg.norm(x) * -get_sign(x[i])
    e = np.zeros(len(x))
    e[i] = 1.0

    u = (x - y * e) / np.linalg.norm(x - y * e)
    H = np.eye(x.shape[0]) - 2 * np.outer(u, u)
    
    return H

In [5]:
def householder_bidiagonalization(A):
    
# returns Bidiagnoliziation of A as (U, A, V.T)
    R = np.copy(A)
    m, n = R.shape
    U = np.eye(m)
    V = np.eye(n)    
    
    for i in range(n - 1):
        
        # zero columns
        h = np.zeros(m)
        h[i:] = R[i:, i]
        H = householder(h, i)
        
        U = np.dot(U, H)
        R = set_lowVal_zero(H @ R)
        

        # zero rows
        h = np.zeros(n)
        h[i+1:] = R[i, i+1:] 
        H1 = householder(h, i + 1)

        V = np.dot(H1, V)
        R = set_lowVal_zero(R @ H1)

    return U, R, V

An important piece of Demmel and Kahan's algorithm is a very efficient way of generating a $2\times2$ "Givens" rotation matrix that annihilates the second component of a vector.

This algorithm computes the cosine $c$ and sine $s$ of a rotation angle that satisfies the following condition.

$$\begin{bmatrix}
        \mathbf{c} & \mathbf{s}\\
         \mathbf{-s} & \mathbf{c}
   \end{bmatrix} 
   \begin{bmatrix}
          \mathbf{f}\\
           \mathbf{g}
    \end{bmatrix} = \begin{bmatrix}
                           \mathbf{r}\\
                            \mathbf{0}
                    \end{bmatrix}$$

In [6]:
def rot(f, g):
    if f == 0:
        c = 0; s = 1
        r = g
    elif np.abs(f) > np.abs(g):
        t = g / f
        t1 = np.sqrt(1 + t*t)
        
        c = 1 / t1; s = t * c
        r = f * t1
    else:
        t = f / g
        t1 = np.sqrt(1 + t*t)
        
        s = 1 / t1; c = t * s
        r = g * t1
        
    return c, s, r

The standard algorithm is based on repeated QR-type "sweeps" through the bidiagonal matrix. In its simplest form, a sweep begins at the top left of the matrix and runs down the rows to the bottom. The algorithm continues until the final step on the bottom row that introduces no non-zeros. This process has been termed "chasing the bulge".

In this algorithm, there are two orthogonal (rotation) matrices, $Q$ , employed.

The important consequence of these two rotations is that the row with three non-zeros in it (the "bulge") has moved from row $i - 1$ to row $i$, while all other rows still have two non-zeros.

In [7]:
def msweep(R):
    n, _ = np.shape(R)
    U = np.eye(n)
    V = np.eye(n)
    
    for i in range(n - 1):
        c, s, r = rot(R[i, i], R[i, i+1])
        # construct matrix Q and multiply on the right by Q.T
        # this annihilates both R(i-1, i+1) and R(i, i+1) 
        # but makes R(i+1, i) non-zero
        Q = np.eye(n)
        Q[i:i+2, i:i+2] = np.array([[c, s], [-s, c]])
        R = set_lowVal_zero(R @ Q.T)
        V = set_lowVal_zero(V @ Q.T)
        
        c, s, r = rot(R[i, i], R[i+1, i])
        # construct matrix Q and multiply on the left by Q
        # This annihilates R(i+1, i) 
        # but makes R(i, i+1) and R(i, i+2) non-zero
        Q = np.eye(n)
        Q[i:i+2, i:i+2] = np.array([[c, s], [-s, c]])
        R = set_lowVal_zero(Q @ R)
        U = set_lowVal_zero(Q @ U)
        
    return U.T, R, V.T

In [ ]:
def svd(X, tol=TOL):
    n, _ = np.shape(X)

    i_upper = n - 1
    i_lower = 0

    U, R, V  = householder_bidiagonalization(X)

    for iter in range(n):
        # find first non-zero bidiagonal element from the end
        for i in range(i_upper, i_lower, -1):
            i_upper = i
            if np.abs(R[i - 1, i]) > TOL:
                break
            else:
                R[i - 1, i] = 0
                
        # find first non-zero bidiagonal element from the begining      
        for j in range(i_lower, i_upper):
            i_lower = j
            if np.abs(R[j, j + 1]) > TOL:
                break
            else:
                R[j, j + 1] = 0
                
        if i_upper == i_lower and np.abs(R[i_upper, i_upper + 1]) <= TOL or i_upper < i_lower:
            break
            
        U1, R, V1 = msweep(R)
        U = np.dot(U, U1)
        V = np.dot(V1, V)
        
    return U, R, V

### Test 1

In [ ]:
np.set_printoptions(suppress=True)

In [ ]:
a = np.array([[4, 3, 0, 2],
              [2, 1, 2, 1],
              [4, 4, 0, 3],
              [5, 6, 1, 3]])


u, s, v = svd(a)

assert_allclose(np.dot(u, u.T), np.eye(4), atol=1e-10)
assert_allclose(np.dot(v, v.T), np.eye(4), atol=1e-10)
assert_allclose(u @ s @ v, a, atol=1e-4)

### Test 2

In [ ]:
rndm = np.random.RandomState(1234)
b = rndm.uniform(size=(5, 5))

u, z, v = svd(b)

assert_allclose(np.dot(u, u.T), np.eye(5), atol=1e-10)
assert_allclose(np.dot(v, v.T), np.eye(5), atol=1e-10)
assert_allclose(u @ z @ v, b, atol=1e-10)

http://www.cs.utexas.edu/users/inderjit/public_papers/HLA_SVD.pdf

http://www.math.pitt.edu/~sussmanm/2071Spring08/lab09/index.html

http://people.duke.edu/~hpgavin/SystemID/References/Golub+Reinsch-NM-1970.pdf